### Libraries

In [1]:
import pandas as pd
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


### API Keys

In [5]:
os.environ["OPENAI_API_KEY"] = "hidden"
os.environ["GOOGLE_API_KEY"] = "hidden"

In [4]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

### Code generation prompts

In [32]:
system_prompt = """
You are a helpful assistant, assinged with generating code to python functions and classes when given a docstring. 
You have vast knowledge of python coding and libraries.
You know and follow the google docstring conventions.
"""

my_prompt = """
Provide a python code to the following docstring: {docstring}

Correct format example:

docstring = 

Iteratively compute the sum of all elements in a list of integers after multiplying each element by its index in the list.
    
Args:
lst (list): a list of integers.

Returns:
int: the sum of all elements in the list after preforming the multipication of the elements by their indices.

solution =
    
def multiply_and_sum(lst):
    res = 0
    for i, item in enumerate(lst):
        res += item * i 
    return res
        
Return the code only, do not include the docstring in your answer. 
"""

### LLM Setup

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_prompt),
])

In [34]:
def generate_code_from_docstring(docstring, chain_solution):
    llm_code = chain_solution.invoke({
        "docstring": docstring
    })
    return llm_code

### OPEN AI

In [35]:
llm_openai = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

output_parser = StrOutputParser()

chain_gpt_turbo = prompt | llm_openai | output_parser

First, generate code for one function and class based on gemini docstring

In [36]:
data = pd.read_csv("data_full_docstrings_generated.csv")
function_2_docstring_gemini = data.loc[2, 'Gemini-1.0-pro']
class_4_docstring_gemini = data.loc[43, 'Gemini-1.0-pro']

In [ ]:
generated_code_function_2_gpt_turbo = generate_code_from_docstring(function_2_docstring_gemini, chain_gpt_turbo)
generated_code_class_4_gpt_turbo = generate_code_from_docstring(class_4_docstring_gemini, chain_gpt_turbo)

print(f"Function 2 generated code: \n{generated_code_function_2_gpt_turbo}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gpt_turbo}")

### GOOGLE

In [37]:
prompt_for_gemini = ChatPromptTemplate.from_template(
    system_prompt + my_prompt
)

llm_google = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-1.0-pro")

chain_gemini = prompt_for_gemini | llm_google | output_parser

In [38]:
generated_code_function_2_gemini = generate_code_from_docstring(function_2_docstring_gemini, chain_gemini)
generated_code_class_4_gemini = generate_code_from_docstring(class_4_docstring_gemini, chain_gemini)

print(f"Function 2 generated code: \n{generated_code_function_2_gemini}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gemini}")


Class 4 generated code: 
class Point_2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = (x**2 + y**2)**0.5
        self.theta = np.arctan2(y, x)

    def __repr__(self):
        return f"Point_2D(x={self.x}, y={self.y})"

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __add__(self, other):
        return Point_2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Point_2D(self.x - other.x, self.y - other.y)

    def distance(self, other):
        return ((self.x - other.x)**2 + (self.y - other.y)**2)**0.5

    def angle_wrt_origin(self, other):
        return np.arccos((self.x * other.x + self.y * other.y) / (self.r * other.r))


Generate code using GPT 4

In [ ]:
llm_openai_gpt4 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-1106-Preview")

output_parser = StrOutputParser()

chain_gpt4 = prompt | llm_openai_gpt4 | output_parser

In [ ]:
generated_code_function_2_gpt4 = generate_code_from_docstring(function_2_docstring_gemini, chain_gpt4)
generated_code_class_4_gpt4 = generate_code_from_docstring(class_4_docstring_gemini, chain_gpt4)

print(f"Function 2 generated code: \n{generated_code_function_2_gpt4}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gpt4}")

Generate code for all models and docstrings

In [ ]:
generated_docstrings_data = pd.read_csv("data_full_docstrings_generated.csv")
generated_docstrings_data.rename({'T5 BaseLine docstring generation': 'T5'}, axis=1, inplace=True)  # column name is too long

models = ['T5', 'Gemini-1.0-pro', 'GPT-3.5 Turbo', 'Claude-instant-1']

tqdm.pandas()

for model_name in models:
    new_code_col = f'generated code for {model_name} docstring'
    generated_docstrings_data[new_code_col] = None
    missing_values_mask = generated_docstrings_data[new_code_col].isna()
    generated_docstrings_data.loc[missing_values_mask, [new_code_col]] = (
        generated_docstrings_data.loc[missing_values_mask, model_name].progress_apply(
            lambda x: generate_code_from_docstring(x, chain_gpt4)
        )
    )
    # Debugging: Print the generated docstrings
    print(generated_docstrings_data.loc[missing_values_mask, [new_code_col]])

generated_docstrings_data.to_csv("code_gen_based_on_docstrings_GPT4.csv", index=False)

### gemini 1.5 flash - free

In [39]:
llm_google_flash = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-1.5-flash")

chain_gemini_flash = prompt_for_gemini | llm_google_flash | output_parser

In [40]:
generated_code_function_2_gemini = generate_code_from_docstring(function_2_docstring_gemini, chain_gemini_flash)
generated_code_class_4_gemini = generate_code_from_docstring(class_4_docstring_gemini, chain_gemini_flash)

print(f"Function 2 generated code: \n{generated_code_function_2_gemini}")
print(f"\nClass 4 generated code: \n{generated_code_class_4_gemini}")

Function 2 generated code: 
```python
def sum_nested_list(lst):
    total = 0
    for item in lst:
        if isinstance(item, (int, float)):
            total += abs(item)
        elif isinstance(item, str):
            total += abs(float(item))
        elif isinstance(item, list):
            total += sum_nested_list(item)
    return total
```

Class 4 generated code: 
```python
import math

class Point_2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.r = math.sqrt(x**2 + y**2)
        self.theta = math.atan2(y, x)

    def __repr__(self):
        return f"Point_2D(x={self.x:.2f}, y={self.y:.2f})"

    def __eq__(self, other):
        if isinstance(other, Point_2D):
            return self.x == other.x and self.y == other.y
        return False

    def __add__(self, other):
        if isinstance(other, Point_2D):
            return Point_2D(self.x + other.x, self.y + other.y)
        return NotImplemented

    def __sub__(self, other):
        if 

In [41]:
generated_docstrings_data = pd.read_csv("data_full_docstrings_generated.csv")
generated_docstrings_data.rename({'T5 BaseLine docstring generation': 'T5'}, axis=1, inplace=True)  # column name is too long

models = ['T5', 'Gemini-1.0-pro', 'GPT-3.5 Turbo', 'Claude-instant-1']

tqdm.pandas()

for model_name in models:
    new_code_col = f'generated code for {model_name} docstring'
    generated_docstrings_data[new_code_col] = None
    missing_values_mask = generated_docstrings_data[new_code_col].isna()
    generated_docstrings_data.loc[missing_values_mask, [new_code_col]] = (
        generated_docstrings_data.loc[missing_values_mask, model_name].progress_apply(
            lambda x: generate_code_from_docstring(x, chain_gemini_flash)
        )
    )
    # Debugging: Print the generated docstrings
    print(generated_docstrings_data.loc[missing_values_mask, [new_code_col]])

generated_docstrings_data.to_csv("code_gen_based_on_docstrings_gemini_flash.csv", index=False)

  0%|          | 0/50 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

                      generated code for T5 docstring
0   ```python\ndef sum_delimited(lst, delimiter):\...
1   ```python\ndef find_the_number_of_changes(s1, ...
2   ```python\ndef sum_nested_structures(structure...
3   ```python\ndef count_words(text):\n  """Gets t...
4   ```python\ndef count_k_columns(matrix, k):\n  ...
5   ```python\ndef get_index_in_tree(tree, target)...
6   ```python\ndef get_half_n_sum_set(n_sum_set, n...
7   ```python\ndef partition_sets_to_subsets(parti...
8   ```python\ndef is_dag(graph):\n    """Determin...
9   ```python\nimport random\nimport datetime\n\nd...
10  ```python\ndef diff_matrices(matrices):\n  """...
11  ```python\ndef longest_subsequence(s1, s2):\n ...
12  ```python\ndef count_mobile_characters(table):...
13  ```python\ndef find_prime_factors(num):\n    "...
14  ```python\ndef find_intersection(graph1, graph...
15  ```python\ndef calculate_subset_sum(target, su...
16  ```python\ndef sum_mult_str(evals):\n  sum_mul...
17  ```python\ndef is_valid_

  0%|          | 0/50 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 32.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<l

...

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

           generated code for GPT-3.5 Turbo docstring
0   ```python\ndef sum_even_indexed_elements(lst):...
1   ```python\ndef count_representations(n, lst):\...
2   ```python\ndef sum_nested_list(lst):\n    tota...
3   ```python\ndef count_decompositions(target, wo...
4   ```python\ndef combinations(n, k):\n  """Calcu...
5   ```python\ndef dfs_level_order(tree, index=0):...
6   ```python\ndef find_subset_with_half_sum(lst):...
7   ```python\ndef stirling_number_second_kind(n, ...
8   ```python\ndef is_dag(graph):\n    def dfs(nod...
9   ```python\ndef recursive_calc(num, x=0):\n    ...
10  ```python\ndef sparse_matrix_difference(lst):\...
11  ```python\ndef longest_increasing_or_decreasin...
12  ```python\ndef median(nums):\n    nums.sort()\...
13  ```python\ndef find_prime_factors(n):\n    fac...
14  ```python\ndef graph_intersection(g1, g2):\n  ...
15  ```python\ndef find_subsets_with_sum(lst, targ...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef check_rep

  0%|          | 0/50 [00:00<?, ?it/s]

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<lo

        generated code for Claude-instant-1 docstring
0   ```python\ndef sum_even(lst):\n    total = 0\n...
1   ```python\ndef count_change(n, lst):\n    if n...
2   ```python\ndef nested_sum(lst):\n    total = 0...
3   ```python\ndef count_decompositions(target, wo...
4   ```python\ndef binomial_coefficient(n, k):\n  ...
5   ```python\ndef dfs_level_order(tree, index=0):...
6   ```python\ndef find_half_sum_subset(lst):\n   ...
7   ```python\ndef count_partitions(n, k):\n    me...
8   ```python\ndef is_dag(graph):\n    def dfs(nod...
9   ```python\ndef recursive_function(num, x=0):\n...
10  ```python\ndef sparse_matrix_difference(lst):\...
11  ```python\ndef longest_subsequence(lst):\n    ...
12  ```python\ndef median(nums):\n    nums = list(...
13  ```python\ndef find_prime_factors(n):\n    fac...
14  ```python\ndef intersection_graph(g1, g2):\n  ...
15  ```python\ndef generate_subsets(lst, target):\...
16  ```python\ndef evaluate_expression(expression)...
17  ```python\ndef check_sub